In [ ]:
import tkinter as tk
from tkinter import messagebox, StringVar, ttk, Scrollbar
import psycopg2
import openpyxl  # Para exportar a .xlsx

# 1. Función para conectar a la base de datos
def conectar_bd():
    try:
        conn = psycopg2.connect(
            dbname="inventario",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432"
        )
        print("Conexión exitosa a la base de datos")
        return conn
    except Exception as e:
        print(f"Error de conexión: {e}")
        messagebox.showerror("Error de Conexión", f"No se pudo conectar a la base de datos: {e}")
        return None

# 2. Función para limpiar los campos del formulario
def limpiar_campos():
    centro_operacion_entry.delete(0, tk.END)
    secuencia_entry.delete(0, tk.END)
    codigo_barras_entry.delete(0, tk.END)
    serie_entry.delete(0, tk.END)
    placa_entry.delete(0, tk.END)
    descripcion_elemento_entry.delete(0, tk.END)
    tipo_parte_entry.delete(0, tk.END)
    tipo_almacenamiento_entry.delete(0, tk.END)
    fecha_ingreso_entry.delete(0, tk.END)
    costo_entry.delete(0, tk.END)
    cedula_resp_entry.set('')  # Limpiar combobox de cédula responsable
    responsable_inventario_entry.set('')  # Limpiar combobox de responsable inventario
    responsable_uso_var.set("")  # Limpiar el responsable de uso
    estado_var.set("")
    ubicacion_var.set("")
    secuencia_busqueda_entry.delete(0, tk.END)  # Limpiar búsqueda de secuencia

# 3. Función para cargar ubicaciones desde la base de datos, ordenadas alfabéticamente
def cargar_ubicaciones():
    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT ubicaciones_litoteca FROM public.ubicaciones_litoteca ORDER BY ubicaciones_litoteca ASC")
            ubicaciones = cursor.fetchall()
            ubicaciones_list = [ubicacion[0] for ubicacion in ubicaciones if ubicacion[0] is not None]
            return ubicaciones_list
        except Exception as e:
            print(f"Error al cargar las ubicaciones: {e}")
            return []
        finally:
            cursor.close()
            conn.close()
    return []

# 4. Función para cargar los responsables de uso desde la base de datos, ordenados alfabéticamente
def cargar_responsables():
    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT nombre FROM public.personal_litoteca ORDER BY nombre ASC")
            responsables = cursor.fetchall()
            responsables_list = [responsable[0] for responsable in responsables if responsable[0] is not None]
            return responsables_list
        except Exception as e:
            print(f"Error al cargar los responsables: {e}")
            return []
        finally:
            cursor.close()
            conn.close()
    return []

# 5. Función para cargar cédulas de responsables desde la base de datos, ordenadas alfabéticamente
def cargar_cedulas_responsables():
    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT cedula FROM public.personal_litoteca ORDER BY cedula ASC")
            cedulas_responsables = cursor.fetchall()
            cedulas_list = [cedula[0] for cedula in cedulas_responsables if cedula[0] is not None]
            return cedulas_list
        except Exception as e:
            print(f"Error al cargar las cédulas: {e}")
            return []
        finally:
            cursor.close()
            conn.close()
    return []

# 6. Función mejorada para búsqueda interactiva en ubicación litoteca
def actualizar_ubicacion(event):
    typed = ubicacion_menu.get()
    if typed == '':
        ubicacion_menu['values'] = ubicaciones_list
    else:
        data = [item for item in ubicaciones_list if typed.lower() in item.lower()]
        ubicacion_menu['values'] = data

# 7. Función mejorada para búsqueda interactiva en responsable de uso
def actualizar_responsable_uso(event):
    typed = responsable_uso_menu.get()
    if typed == '':
        responsable_uso_menu['values'] = responsables_list
    else:
        data = [item for item in responsables_list if typed.lower() in item.lower()]
        responsable_uso_menu['values'] = data

# 8. Función mejorada para búsqueda interactiva en cédula de responsable
def actualizar_cedula_responsable(event):
    typed = cedula_resp_entry.get()
    if typed == '':
        cedula_resp_entry['values'] = cedulas_responsables_list
    else:
        data = [item for item in cedulas_responsables_list if typed.lower() in item.lower()]
        cedula_resp_entry['values'] = data

# 9. Función mejorada para búsqueda interactiva en responsable de inventario
def actualizar_responsable_inventario(event):
    typed = responsable_inventario_entry.get()
    if typed == '':
        responsable_inventario_entry['values'] = responsables_list
    else:
        data = [item for item in responsables_list if typed.lower() in item.lower()]
        responsable_inventario_entry['values'] = data

# 10. Función para buscar un registro por Código de Barras y actualizar 'verificado' a TRUE automáticamente
def buscar_por_codigo_barras():
    codigo_barras = codigo_barras_busqueda_entry.get()
    if len(codigo_barras) < 6:
        return  # Si no hay suficientes caracteres, no realizar búsqueda

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("""
                SELECT iv.centro_operacion, iv.secuencia, iv.codigo_barras, 
                       iv.serie, iv.placa, iv.descripcion_elemento, iv.tipo_parte, 
                       iv.tipo_almacenamiento, iv.fecha_ingreso, iv.costo, iv.cedula_resp,
                       iv.responsable_inventario, iv.responsable_uso, iv.estado_elemento, 
                       iv.ub_verificada_2024, iv.verificado
                FROM inventario_verificado iv
                WHERE iv.codigo_barras = %s;
            """, (codigo_barras,))
            resultado = cursor.fetchone()

            if resultado:
                limpiar_campos()  # Limpiar campos antes de cargar los nuevos datos
                centro_operacion_entry.insert(0, resultado[0] if resultado[0] else "N/A")
                secuencia_entry.insert(0, resultado[1] if resultado[1] else "N/A")
                codigo_barras_entry.insert(0, resultado[2] if resultado[2] else "N/A")
                serie_entry.insert(0, resultado[3] if resultado[3] else "N/A")
                placa_entry.insert(0, resultado[4] if resultado[4] else "N/A")
                descripcion_elemento_entry.insert(0, resultado[5] if resultado[5] else "N/A")
                tipo_parte_entry.insert(0, resultado[6] if resultado[6] else "N/A")
                tipo_almacenamiento_entry.insert(0, resultado[7] if resultado[7] else "N/A")
                fecha_ingreso_entry.insert(0, resultado[8] if resultado[8] else "N/A")
                costo_entry.insert(0, resultado[9] if resultado[9] else "N/A")
                cedula_resp_entry.set(resultado[10] if resultado[10] else "")
                responsable_inventario_entry.set(resultado[11] if resultado[11] else "")
                responsable_uso_var.set(resultado[12] if resultado[12] else "")
                estado_var.set(resultado[13] if resultado[13] else "N/A")
                ubicacion_var.set(resultado[14] if resultado[14] else "")

                # Limpiar búsqueda de secuencia antes de copiar la nueva secuencia
                secuencia_busqueda_entry.delete(0, tk.END)
                secuencia_busqueda_entry.insert(0, resultado[1] if resultado[1] else "")

                # Actualizar automáticamente el campo 'verificado' a TRUE en la base de datos
                cursor.execute("""
                    UPDATE inventario_verificado
                    SET verificado = TRUE
                    WHERE codigo_barras = %s;
                """, (codigo_barras,))
                conn.commit()

                # Registrar búsqueda en la tabla de resultados
                registrar_busqueda(resultado[2], resultado[1], resultado[3], resultado[4], resultado[5], resultado[13], resultado[14], resultado[12])

                # Limpiar solo el campo de búsqueda del código de barras
                codigo_barras_busqueda_entry.delete(0, tk.END)

            else:
                limpiar_campos()  # Limpiar campos si no encuentra el código de barras
                messagebox.showinfo("Información", "No se encontró ningún registro con ese Código de Barras.")
        except Exception as e:
            print(f"Error al ejecutar la consulta: {e}")
        finally:
            cursor.close()
            conn.close()

# 11. Función para registrar las búsquedas en la tabla
def registrar_busqueda(codigo_barras, secuencia, serie, placa, descripcion, estado, ubicacion, responsable):
    # Verificar si se ha seleccionado una ubicación
    if not ubicacion_var.get():
        messagebox.showwarning("Advertencia", "Seleccione la ubicación del elemento")
        return

    # Asegurar que el código de barras se almacene como cadena para preservar los ceros a la izquierda
    codigo_barras_str = str(codigo_barras)

    # Añadir los resultados de la búsqueda a la tabla
    tree.insert('', 'end', values=(codigo_barras_str, secuencia, serie, placa, descripcion, estado, ubicacion, responsable))

# 12. Función para exportar los resultados de búsqueda a un archivo .xlsx (manteniendo los ceros a la izquierda)
def exportar_consulta():
    archivo = 'consulta_sesion.xlsx'

    # Crear el archivo de Excel con el libro de trabajo
    libro = openpyxl.Workbook()

    # Seleccionar la hoja activa
    hoja = libro.active
    hoja.title = "Resultados de Búsqueda"

    # Escribir las cabeceras de la tabla
    cabeceras = ["Código de Barras", "Secuencia", "Serie", "Placa", "Descripción", "Estado", "Ubicación", "Responsable de Uso"]
    hoja.append(cabeceras)  # Escribir las cabeceras

    # Escribir los datos de la tabla
    for row_id in tree.get_children():
        valores = tree.item(row_id)['values']
        
        # Convertir cada valor a texto explícitamente y asegurarse de que los ceros a la izquierda se conserven
        valores_texto = [str(valor).zfill(6) if isinstance(valor, int) or valor.isdigit() else str(valor) for valor in valores]
        hoja.append(valores_texto)  # Agregar cada fila como texto

    # Guardar el archivo
    libro.save(archivo)
    messagebox.showinfo("Exportar", f"Consulta exportada a {archivo} exitosamente.")

# 13. Función para insertar un nuevo registro
def insertar_nuevo_registro():
    # Obtener datos ingresados en los campos
    centro_operacion = centro_operacion_entry.get()
    secuencia = secuencia_entry.get()
    codigo_barras = codigo_barras_entry.get()
    serie = serie_entry.get()
    placa = placa_entry.get()
    descripcion = descripcion_elemento_entry.get()
    tipo_parte = tipo_parte_entry.get()
    tipo_almacenamiento = tipo_almacenamiento_entry.get()
    fecha_ingreso = fecha_ingreso_entry.get()
    costo = costo_entry.get()
    cedula_resp = cedula_resp_entry.get()
    responsable_inventario = responsable_inventario_entry.get()
    responsable_uso = responsable_uso_var.get()
    estado_elemento = estado_var.get()
    ubicacion = ubicacion_var.get()

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            # Insertar un nuevo registro con todos los campos
            cursor.execute("""
                INSERT INTO inventario_verificado (centro_operacion, secuencia, codigo_barras, serie, placa, descripcion_elemento, tipo_parte, tipo_almacenamiento, fecha_ingreso, costo, cedula_resp, responsable_inventario, responsable_uso, estado_elemento, ub_verificada_2024, verificado)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, FALSE);
            """, (centro_operacion, secuencia, codigo_barras, serie, placa, descripcion, tipo_parte, tipo_almacenamiento, fecha_ingreso, costo, cedula_resp, responsable_inventario, responsable_uso, estado_elemento, ubicacion))
            conn.commit()

            # Cargar el nuevo código de barras en el campo correspondiente
            codigo_barras_entry.insert(0, codigo_barras)
            messagebox.showinfo("Nuevo Registro", "Se ha insertado un nuevo registro exitosamente.")
        except Exception as e:
            conn.rollback()
            print(f"Error al insertar el nuevo registro: {e}")
        finally:
            cursor.close()
            conn.close()

# 14. Función para guardar los cambios realizados en el formulario o insertar un nuevo registro
def guardar_cambios():
    codigo_barras = codigo_barras_entry.get()
    centro_operacion = centro_operacion_entry.get()
    secuencia = secuencia_entry.get()
    serie = serie_entry.get()
    placa = placa_entry.get()
    descripcion = descripcion_elemento_entry.get()
    tipo_parte = tipo_parte_entry.get()
    tipo_almacenamiento = tipo_almacenamiento_entry.get()
    fecha_ingreso = fecha_ingreso_entry.get()
    costo = costo_entry.get()
    cedula_resp = cedula_resp_entry.get()
    responsable_inventario = responsable_inventario_entry.get()
    responsable_uso = responsable_uso_var.get()
    estado_elemento = estado_var.get()
    ubicacion = ubicacion_var.get()

    if not codigo_barras:
        messagebox.showwarning("Advertencia", "No se ha seleccionado ningún registro para actualizar.")
        return

    if estado_elemento not in ["Bueno", "Inservible"]:
        messagebox.showwarning("Advertencia", "El estado debe ser 'Bueno' o 'Inservible'.")
        return

    if ubicacion not in ubicaciones_list:
        messagebox.showwarning("Advertencia", "Por favor, seleccione una ubicación válida.")
        return

    if responsable_uso not in responsables_list:
        messagebox.showwarning("Advertencia", "Por favor, seleccione un responsable de uso válido.")
        return

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            # Verificar si el registro ya existe
            cursor.execute("SELECT COUNT(*) FROM inventario_verificado WHERE codigo_barras = %s;", (codigo_barras,))
            existe = cursor.fetchone()[0]

            if existe:
                # Actualizar el registro existente con todos los campos
                cursor.execute("""
                    UPDATE inventario_verificado
                    SET centro_operacion = %s, secuencia = %s, serie = %s, placa = %s, descripcion_elemento = %s, 
                        tipo_parte = %s, tipo_almacenamiento = %s, fecha_ingreso = %s, costo = %s, cedula_resp = %s, 
                        responsable_inventario = %s, responsable_uso = %s, estado_elemento = %s, ub_verificada_2024 = %s, verificado = TRUE
                    WHERE codigo_barras = %s;
                """, (centro_operacion, secuencia, serie, placa, descripcion, tipo_parte, tipo_almacenamiento, fecha_ingreso, costo, cedula_resp, responsable_inventario, responsable_uso, estado_elemento, ubicacion, codigo_barras))
                conn.commit()
                messagebox.showinfo("Éxito", "Registro actualizado exitosamente.")
            else:
                # Insertar un nuevo registro si no existe
                cursor.execute("""
                    INSERT INTO inventario_verificado (codigo_barras, estado_elemento, ub_verificada_2024, responsable_uso, verificado)
                    VALUES (%s, %s, %s, %s, TRUE);
                """, (codigo_barras, estado_elemento, ubicacion, responsable_uso))
                conn.commit()
                messagebox.showinfo("Éxito", "Nuevo registro insertado exitosamente.")

        except Exception as e:
            conn.rollback()
            print(f"Error al guardar los cambios: {e}")
            messagebox.showerror("Error", f"No se pudieron guardar los cambios: {e}")
        finally:
            cursor.close()
            conn.close()

# 15. Función para buscar un registro por Secuencia y mostrar resultados en el Frame Derecho
def buscar_por_secuencia():
    secuencia = secuencia_busqueda_entry.get()
    if not secuencia:
        messagebox.showwarning("Advertencia", "Por favor, ingrese una Secuencia.")
        return

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("""
                SELECT iv.secuencia, iv.codigo_barras, iv.serie, iv.placa, iv.descripcion_elemento, 
                       iv.estado_elemento, iv.ub_verificada_2024, iv.tipo_parte
                FROM inventario_verificado iv
                WHERE iv.secuencia = %s;
            """, (secuencia,))
            resultado = cursor.fetchall()

            if resultado:
                # Limpiar el frame de resultados antes de mostrar los nuevos datos
                for widget in result_frame.winfo_children():
                    widget.destroy()

                for res in resultado:
                    result_text = f"Secuencia: {res[0]}\nCódigo de Barras: {res[1]}\nSerie: {res[2]}\nPlaca: {res[3]}\nDescripción: {res[4]}\nEstado: {res[5]}\nUbicación: {res[6]}\nParte: {res[7]}\n" + "-"*30
                    tk.Label(result_frame, text=result_text, justify="left", anchor="w").pack(fill="both")
            else:
                # Limpiar el frame de resultados si no encuentra la secuencia
                for widget in result_frame.winfo_children():
                    widget.destroy()
                tk.Label(result_frame, text="No se encontró ningún registro con esa Secuencia.").pack()

        except Exception as e:
            print(f"Error al buscar por secuencia: {e}")
        finally:
            cursor.close()
            conn.close()

# 16. Configuración de la ventana principal de Tkinter
root = tk.Tk()
root.title("Verificación Inventario Bienes Devolutivos")

# Crear los frames para dividir la pantalla
frame_left = tk.Frame(root)
frame_left.grid(row=0, column=0, padx=20, pady=20, sticky="n")

frame_right = tk.Frame(root)
frame_right.grid(row=0, column=1, padx=20, pady=20, sticky="n")

# Búsqueda por código de barras
tk.Label(frame_left, text="Buscar por Código de Barras:").grid(row=0, column=0, sticky="w")
codigo_barras_busqueda_entry = tk.Entry(frame_left, width=50)
codigo_barras_busqueda_entry.grid(row=0, column=1, padx=5)
codigo_barras_busqueda_entry.bind("<KeyRelease>", lambda event: buscar_por_codigo_barras())  # Búsqueda automática tras 6 caracteres

tk.Button(frame_left, text="Buscar", command=buscar_por_codigo_barras).grid(row=0, column=2, padx=5)

# Campos del formulario con ancho de 50
centro_operacion_entry = tk.Entry(frame_left, width=50)
secuencia_entry = tk.Entry(frame_left, width=50)
codigo_barras_entry = tk.Entry(frame_left, width=50)
serie_entry = tk.Entry(frame_left, width=50)
placa_entry = tk.Entry(frame_left, width=50)
descripcion_elemento_entry = tk.Entry(frame_left, width=50)
tipo_parte_entry = tk.Entry(frame_left, width=50)
tipo_almacenamiento_entry = tk.Entry(frame_left, width=50)
fecha_ingreso_entry = tk.Entry(frame_left, width=50)
costo_entry = tk.Entry(frame_left, width=50)
cedula_resp_entry = ttk.Combobox(frame_left, textvariable=StringVar(), values=cargar_cedulas_responsables(), width=47)  # Combobox de cédula responsable
responsable_inventario_entry = ttk.Combobox(frame_left, textvariable=StringVar(), values=cargar_responsables(), width=47)  # Combobox responsable de inventario

# Variables para Combobox
ubicacion_var = StringVar()
responsable_uso_var = StringVar()

# Agregamos las etiquetas y campos de entrada de manera explícita
campos = [
    ("Centro Operación", centro_operacion_entry),
    ("Secuencia", secuencia_entry),
    ("Código de Barras", codigo_barras_entry),
    ("Serie", serie_entry),
    ("Placa", placa_entry),
    ("Descripción Elemento", descripcion_elemento_entry),
    ("Tipo Parte", tipo_parte_entry),
    ("Tipo Almacenamiento", tipo_almacenamiento_entry),
    ("Fecha Ingreso", fecha_ingreso_entry),
    ("Costo", costo_entry),
    ("Cédula Responsable", cedula_resp_entry),
    ("Responsable Inventario", responsable_inventario_entry)
]

for i, (label_text, entry) in enumerate(campos):
    tk.Label(frame_left, text=f"{label_text}:").grid(row=i + 1, column=0, sticky="w")
    entry.grid(row=i + 1, column=1, columnspan=2, padx=5, pady=2)

# Menú desplegable para estado
tk.Label(frame_left, text="Estado Elemento:").grid(row=len(campos) + 1, column=0, sticky="w")
estado_var = StringVar(frame_left)
estado_menu = ttk.Combobox(frame_left, textvariable=estado_var, values=["Bueno", "Inservible"], width=47)
estado_menu.grid(row=len(campos) + 1, column=1, columnspan=2, padx=5)

# Menú desplegable para el campo "Ubicación Litoteca"
tk.Label(frame_left, text="Ubicación Litoteca:").grid(row=len(campos) + 2, column=0, sticky="w")
ubicacion_menu = ttk.Combobox(frame_left, textvariable=ubicacion_var, values=cargar_ubicaciones(), width=47)  # Incremento en 25%
ubicacion_menu.grid(row=len(campos) + 2, column=1, columnspan=2, padx=5)
ubicacion_menu.bind("<KeyRelease>", actualizar_ubicacion)  # Búsqueda interactiva en ubicación

# Menú desplegable para el campo "Responsable de Uso"
tk.Label(frame_left, text="Responsable de Uso:").grid(row=len(campos) + 3, column=0, sticky="w")
responsable_uso_menu = ttk.Combobox(frame_left, textvariable=responsable_uso_var, values=cargar_responsables(), width=47)  # Incremento en 25%
responsable_uso_menu.grid(row=len(campos) + 3, column=1, columnspan=2, padx=5)
responsable_uso_menu.bind("<KeyRelease>", actualizar_responsable_uso)  # Búsqueda interactiva en responsables

# Combobox de búsqueda interactiva para Cédula Responsable
cedula_resp_entry.bind("<KeyRelease>", actualizar_cedula_responsable)  # Búsqueda interactiva en cédula responsable

# Combobox de búsqueda interactiva para Responsable de Inventario
responsable_inventario_entry.bind("<KeyRelease>", actualizar_responsable_inventario)  # Búsqueda interactiva en responsable de inventario

# Botones de control
tk.Button(frame_left, text="Nuevo Registro", command=lambda: [insertar_nuevo_registro(), cargar_responsable()]).grid(row=len(campos) + 4, column=0, padx=5, pady=10)
tk.Button(frame_left, text="Limpiar Campos", command=limpiar_campos).grid(row=len(campos) + 4, column=1, padx=5, pady=10)
tk.Button(frame_left, text="Guardar Cambios", command=guardar_cambios).grid(row=len(campos) + 4, column=2, padx=5, pady=10)

# Botón para exportar los resultados de búsqueda
tk.Button(frame_left, text="Exportar Consulta", command=exportar_consulta).grid(row=len(campos) + 5, column=1, padx=5, pady=10)

# ------------------ Frame Derecho: Búsqueda por Secuencia ------------------
tk.Label(frame_right, text="Buscar por Secuencia:").grid(row=0, column=0, sticky="w")
secuencia_busqueda_entry = tk.Entry(frame_right)
secuencia_busqueda_entry.grid(row=0, column=1, padx=5)
tk.Button(frame_right, text="Buscar", command=buscar_por_secuencia).grid(row=0, column=2, padx=5)

# Frame para los resultados de búsqueda por secuencia
result_frame = tk.Frame(frame_right)
result_frame.grid(row=1, column=0, columnspan=3, pady=10)

# ---------------- Tabla para registrar las búsquedas realizadas con Scrollbar ----------------
scrollbar = Scrollbar(root)
scrollbar.grid(row=1, column=2, sticky='ns')

columns = ("Código de Barras", "Secuencia", "Serie", "Placa", "Descripción", "Estado", "Ubicación", "Responsable de Uso")
tree = ttk.Treeview(root, columns=columns, show="headings", yscrollcommand=scrollbar.set)
scrollbar.config(command=tree.yview)

tree.heading("Código de Barras", text="Código de Barras")
tree.heading("Secuencia", text="Secuencia")
tree.heading("Serie", text="Serie")
tree.heading("Placa", text="Placa")
tree.heading("Descripción", text="Descripción")
tree.heading("Estado", text="Estado")
tree.heading("Ubicación", text="Ubicación")
tree.heading("Responsable de Uso", text="Responsable de Uso")
tree.grid(row=1, column=0, columnspan=2, padx=20, pady=20)

# Copyright y finalización
copyright_label = tk.Label(root, text="© 2024 Jorge A Melo", font=("Arial", 10))
copyright_label.grid(row=2, column=0, columnspan=2, pady=10)

# Cargar ubicaciones al iniciar la aplicación
ubicaciones_list = cargar_ubicaciones()
responsables_list = cargar_responsables()
cedulas_responsables_list = cargar_cedulas_responsables()

# Ejecutar la aplicación
root.mainloop()

NameError: name 'func_3_cargar_lista' is not defined